In [1]:
from pathlib import Path
from dateutil.parser import parse as parse_date

In [2]:
project_name = 'CA_144_476'
work_dir = Path.cwd() / project_name
data_dir = work_dir / 'data'

stack_start = parse_date('2017-08-01 00:00:00Z')
stack_end = parse_date('2022-03-11 00:00:00Z')
max_temporal_baseline = 30 #days

data_dir.mkdir(parents=True, exist_ok=True)

In [3]:
import asf_search as asf

In [4]:
import pandas as pd

In [5]:
# filter the Sentinel-1B data
aoi = 'POLYGON((-121.9936 33.1227,-117.5164 33.1227,-117.5164 36.191,-121.9936 36.191,-121.9936 33.1227))'
search_results = asf.geo_search(
        platform=asf.PLATFORM.SENTINEL1A,
        intersectsWith=aoi,
        start='2017-08-01',
        end='2022-03-11',
        processingLevel=asf.PRODUCT_TYPE.SLC,
        beamSwath='IW',
        flightDirection=asf.FLIGHT_DIRECTION.DESCENDING,
        relativeOrbit=144,
        asfFrame=476 
    )

/home/jovyan/.local/envs/opensarlab_mintpy_recipe_book-3/lib/python3.9/site-packages/tzlocal/unix.py:193: UserWarning: Can not find any timezone configuration, defaulting to UTC.
  warnings.warn("Can not find any timezone configuration, defaulting to UTC.")


In [6]:
with open("search_results1.csv", "w") as f:
   f.writelines(search_results.csv())

In [7]:
baseline_results = asf.baseline_search.stack_from_product(search_results[-1])

columns = list(baseline_results[0].properties.keys()) + ['geometry', ]
data = [list(scene.properties.values()) + [scene.geometry, ] for scene in baseline_results]

stack = pd.DataFrame(data, columns=columns)
stack['startTime'] = stack.startTime.apply(parse_date)

stack = stack.loc[(stack_start <= stack.startTime) & (stack.startTime <= stack_end)]

In [8]:
# somewhat "stack" contains Sentinel-1B data
stack.to_csv('stack.csv',index=False)

In [9]:
# drop column content equal to Sentinel-1B
stack_filtered = stack[stack['granuleType'] != 'SENTINEL_1B_FRAME']
stack_filtered.to_csv('stack_filtered.csv',index=False)

In [12]:
sbas_pairs = set()

for reference, rt in stack_filtered.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack_filtered.loc[
        (stack_filtered.sceneName != reference)
        & (stack_filtered.temporalBaseline - rt <= max_temporal_baseline)
        & (stack_filtered.temporalBaseline - rt > 0)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))

In [13]:
len(sbas_pairs)

269

In [23]:
import hyp3_sdk as sdk
hyp3 = sdk.HyP3(prompt=True)

NASA Earthdata Login username:  vickielee333
NASA Earthdata Login password:  ········


In [24]:
jobs = sdk.Batch()
for reference, secondary in sbas_pairs:
    jobs += hyp3.submit_insar_job(reference, secondary, name=project_name,
                                  include_dem=True, include_look_vectors=True)

In [25]:
jobs = hyp3.watch(jobs)

  0%|          | 0/829 [timeout in 10800 s]

ServerError: 